## Setup

### Imports

In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 

### Notebook settings

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
# pd.set_option('display.max_colwidth', None)

sns.set_theme(style='white', palette='Blues_r')

### Functions

In [3]:
# Function to replace ' ' with '_'
def fix_headers(df, remove=' ', replace='_'):
    return [header.replace(remove, replace).lower() for header in df.columns.to_list()]


### Reading in data

In [4]:
# Read in files
df_actual = pd.read_excel(
    '../04.input/Actuals_forecasts_Scenario_planning_test_DM01v2_hatch (1).xlsx', header=0)
df_weeks_wait = pd.read_excel(
    '../04.input/Hatch_Weeks Wait by Diagnostic Sub-Category.xlsx', header=1)
CAPACITY = 1394

In [5]:
df_actual.head()

,week,metric,Sum of actuals,Sum of forecast,Sum of conf_lower,Sum of conf_upper,Sum of trend
0,2021-08-01 00:00:00,Breaches,209.0,274.48,0.000000,654.704154,632.135112
1,2021-08-01 00:00:00,Waiting List,3726.8,3269.32,3074.735168,3461.990447,193.947977
2,2021-08-08 00:00:00,Breaches,179.3,299.86,0.000000,705.534084,637.961510
3,2021-08-08 00:00:00,Waiting List,3936.9,3278.72,3085.400910,3467.190201,193.418329
4,2021-08-15 00:00:00,Breaches,235.4,306.44,0.000000,738.123355,643.787907


In [6]:
df_weeks_wait.head()

,Unnamed: 0,Diagnostic Sub-Category,Diagnostic Description,Count of Diagnostics_ID,Current_Wait_band
0,NaN,Barium Enema,Endoscopic US pancreas,1,2
1,NaN,Barium Enema,Endoscopic US pancreas,5,19
2,NaN,Barium Enema,Endoscopic US pancreas,4,21
3,NaN,Barium Enema,Endoscopic US stomach,1,7
4,NaN,Cardiac MRI,MRI,2,1


## Cleaning Input Data

### Cleaning df_actual

In [7]:
df_actual = df_actual[['week', 'metric', 'Sum of actuals']][:-1].copy()
df_actual['week'] = pd.to_datetime(df_actual['week'])

In [8]:
# Create columns for each of the three metrics
df_actual = df_actual.pivot(columns='metric', index='week', values='Sum of actuals').copy()

In [9]:
df_actual.columns = fix_headers(df_actual)

In [10]:
# Create an dataset for univariate forecasting for new demand, breaches and waiting list that includes all dates
df_actual_for_forecasting = df_actual.copy()
df_actual_for_forecasting = df_actual_for_forecasting[df_actual_for_forecasting['new_demand'] != 0].copy()

In [11]:
df_actual_for_forecasting.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 226 entries, 2019-09-15 to 2024-01-14
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   breaches      226 non-null    float64
 1   new_demand    94 non-null     float64
 2   waiting_list  226 non-null    float64
dtypes: float64(3)
memory usage: 7.1 KB


In [12]:
# Ensure there are no missing dates
start_date = df_actual_for_forecasting.index.min()
end_date = df_actual_for_forecasting.index.max()
start_date, end_date

(Timestamp('2019-09-15 00:00:00'), Timestamp('2024-01-14 00:00:00'))

In [13]:
# Create a series containing all the dates in the specified range
weeks_in_info = pd.date_range(start_date, end_date, freq='W')

In [14]:
# Find if any weeks are not in the forecast dataframe
missing_weeks = [week for week in weeks_in_info.tolist() if week not in df_actual_for_forecasting.index]
missing_weeks

[Timestamp('2020-10-18 00:00:00')]

- missing week 2020-10-18

In [15]:
# Add in the missing week and interpolate the missing data
df_actual_for_forecasting = df_actual_for_forecasting.reindex(weeks_in_info)
df_actual_for_forecasting = df_actual_for_forecasting.interpolate(limit=1).copy()

In [16]:
# Create series for each actual metric
s_new_demand_for_forecasting = df_actual_for_forecasting['new_demand'].dropna().copy()
s_breaches_for_forecasting = df_actual_for_forecasting['breaches'].copy()
s_waiting_list_for_forecasting = df_actual_for_forecasting['waiting_list'].copy()

In [17]:
# Create features to be used in EDA
df_actual['breaches_delta'] = df_actual['breaches'].diff()
df_actual['waiting_list_delta'] = df_actual['waiting_list'].diff()
df_actual['new_demand_delta'] = df_actual['new_demand'].diff()
df_actual['scans_done'] = df_actual['new_demand'] - df_actual['waiting_list_delta'].shift(-1)
df_actual['percent_breaches'] = df_actual['breaches'] / df_actual['waiting_list']
df_actual['capacity'] = CAPACITY
df_actual[df_actual.index > '2023-01-01']

,breaches,new_demand,waiting_list,breaches_delta,waiting_list_delta,new_demand_delta,scans_done,percent_breaches,capacity
week,,,,,,,,,
2023-01-08,170.5,1039.5,3617.9,92.4,-7.7,272.8,1133.0,0.047127,1394
2023-01-15,183.7,1138.5,3524.4,13.2,-93.5,99.0,1036.2,0.052122,1394
2023-01-22,215.6,1378.3,3626.7,31.9,102.3,239.8,1425.6,0.059448,1394
2023-01-29,139.7,1390.4,3579.4,-75.9,-47.3,12.1,1478.4,0.039029,1394
2023-02-05,93.5,1282.6,3491.4,-46.2,-88.0,-107.8,1105.5,0.026780,1394
2023-02-12,78.1,1403.6,3668.5,-15.4,177.1,121.0,1494.9,0.021289,1394
2023-02-19,78.1,1223.2,3577.2,0.0,-91.3,-180.4,1126.4,0.021833,1394
2023-02-26,57.2,1291.4,3674.0,-20.9,96.8,68.2,1174.8,0.015569,1394
2023-03-05,51.7,1333.2,3790.6,-5.5,116.6,41.8,1249.6,0.013639,1394


In [18]:
# Filter out results with no value for new demand
df_actual = df_actual[(df_actual['new_demand'].notna()) & (df_actual['new_demand'] != 0)].copy()
df_actual.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 94 entries, 2022-04-03 to 2024-01-14
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   breaches            94 non-null     float64
 1   new_demand          94 non-null     float64
 2   waiting_list        94 non-null     float64
 3   breaches_delta      94 non-null     float64
 4   waiting_list_delta  94 non-null     float64
 5   new_demand_delta    93 non-null     float64
 6   scans_done          94 non-null     float64
 7   percent_breaches    94 non-null     float64
 8   capacity            94 non-null     int64  
dtypes: float64(8), int64(1)
memory usage: 7.3 KB


In [19]:
# Fill na with 0 as there is no change in new demand for the first entry
df_actual['new_demand_delta'] = df_actual['new_demand_delta'].fillna(0)
df_actual.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 94 entries, 2022-04-03 to 2024-01-14
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   breaches            94 non-null     float64
 1   new_demand          94 non-null     float64
 2   waiting_list        94 non-null     float64
 3   breaches_delta      94 non-null     float64
 4   waiting_list_delta  94 non-null     float64
 5   new_demand_delta    94 non-null     float64
 6   scans_done          94 non-null     float64
 7   percent_breaches    94 non-null     float64
 8   capacity            94 non-null     int64  
dtypes: float64(8), int64(1)
memory usage: 7.3 KB


- df_actual is now clean

### Cleaning df_weeks_wait

In [20]:
df_weeks_wait = df_weeks_wait.drop(['Unnamed: 0'], axis=1).copy()
df_weeks_wait

,Diagnostic Sub-Category,Diagnostic Description,Count of Diagnostics_ID,Current_Wait_band
0,Barium Enema,Endoscopic US pancreas,1,2
1,Barium Enema,Endoscopic US pancreas,5,19
2,Barium Enema,Endoscopic US pancreas,4,21
3,Barium Enema,Endoscopic US stomach,1,7
4,Cardiac MRI,MRI,2,1
5,Cardiac MRI,MRI,8,2
6,Cardiac MRI,MRI,9,3
7,Cardiac MRI,MRI,1,4
8,Cardiac MRI,MRI,8,5
9,Cardiac MRI,MRI,13,6


In [21]:
df_weeks_wait.columns = fix_headers(df_weeks_wait)
df_weeks_wait.columns = fix_headers(df_weeks_wait, remove='-')
df_weeks_wait

,diagnostic_sub_category,diagnostic_description,count_of_diagnostics_id,current_wait_band
0,Barium Enema,Endoscopic US pancreas,1,2
1,Barium Enema,Endoscopic US pancreas,5,19
2,Barium Enema,Endoscopic US pancreas,4,21
3,Barium Enema,Endoscopic US stomach,1,7
4,Cardiac MRI,MRI,2,1
5,Cardiac MRI,MRI,8,2
6,Cardiac MRI,MRI,9,3
7,Cardiac MRI,MRI,1,4
8,Cardiac MRI,MRI,8,5
9,Cardiac MRI,MRI,13,6


In [22]:
# Creating complete dataset without CT filter
df_weeks_wait_all = df_weeks_wait.copy()

In [23]:
# Filter df_weeks_wait to only have CT scans
ct_filter = (df_weeks_wait['diagnostic_sub_category'].str.contains('CT')) \
    | (df_weeks_wait['diagnostic_description'].str.contains('CT'))
df_weeks_wait = df_weeks_wait[ct_filter].reset_index(drop=True)

In [24]:
df_weeks_wait

,diagnostic_sub_category,diagnostic_description,count_of_diagnostics_id,current_wait_band
0,CT,CT Abdomen,6,0
1,CT,CT Abdomen,2,1
2,CT,CT Abdomen,7,2
3,CT,CT Abdomen,3,3
4,CT,CT Abdomen,7,4
5,CT,CT Abdomen,4,6
6,CT,CT Abdomen,5,8
7,CT,CT Abdomen,1,11
8,CT,CT Abdomen,4,12
9,CT,CT Abdomen,4,17


In [25]:
# Creating column to show whether there is a breach (wait band > 6)
df_weeks_wait['is_breach'] = df_weeks_wait['current_wait_band'].apply(lambda x: 'Y' if x > 6 else 'N')
df_weeks_wait

,diagnostic_sub_category,diagnostic_description,count_of_diagnostics_id,current_wait_band,is_breach
0,CT,CT Abdomen,6,0,N
1,CT,CT Abdomen,2,1,N
2,CT,CT Abdomen,7,2,N
3,CT,CT Abdomen,3,3,N
4,CT,CT Abdomen,7,4,N
5,CT,CT Abdomen,4,6,N
6,CT,CT Abdomen,5,8,Y
7,CT,CT Abdomen,1,11,Y
8,CT,CT Abdomen,4,12,Y
9,CT,CT Abdomen,4,17,Y


- df_weeks_wait is now clean

## Write Data Frames to csv

In [26]:
# Write data frames to csv
filenames = [
    'actual', 
    'weeks_wait', 
    'weeks_wait_all',
    'actual_for_forecasting',
    'breaches_for_forecasting',
    'new_demand_for_forecasting',
    'waiting_list_for_forecasting',
    ]
dfs = [
    df_actual, 
    df_weeks_wait, 
    df_weeks_wait_all, 
    df_actual_for_forecasting, 
    s_breaches_for_forecasting, 
    s_new_demand_for_forecasting, 
    s_waiting_list_for_forecasting
    ]
for df, name in zip(dfs, filenames):
    df.to_csv(f'../05.output/actual/{name}.csv')